In [66]:
import time
import email
import smtplib
import imaplib
import mailparser
import mysql
import mysql.connector


conn=mysql.connector.connect(host="localhost",user="root",password="abcd@1234")
crs=conn.cursor()
crs.execute('use mpr')
insert='insert into db values(null,%s,%s,%s,%s,%s)'

def read_email_from_gmail(crs):
    try:
        server = imaplib.IMAP4_SSL('imap.gmail.com')
        server.login("garglakshay631@gmail.com","abcd@1234")
        server.select('inbox')

        typ, data = server.search(None, 'ALL')
        print("Connection",typ)
        print(data)
        mail_ids = data[0]
        
        id_list = mail_ids.split()  
        first_email = int(id_list[0])
        latest_email = int(id_list[-1])
#         typ, data = server.fetch(str(i), '(RFC822)' )
#         mail=mailparser.parse_from_bytes(data[-1])
        for i in range(latest_email,first_email-1,-1):
            typ, data = server.fetch(str(i), '(RFC822)' )
            for response_part in data:
                if isinstance(response_part, tuple):
                    mail = mailparser.parse_from_bytes(response_part[-1])
                    name=mail.subject
                    to=mail.to[-1][-1]
                    from_=mail.from_[-1][-1]
                    att=mail.attachments
                    body=mail.text_plain[-1]
                    b=body.split('\r\n')
                    print(from_,to,name,b)
                    val=(from_,to,name,)
                    if(len(att)==0):
                        print(from_,to,name,"Please ask the organizer to send the poster")
                    else:
                        print(from_,to,name,att[-1],type(att[-1]))
                    crs.execute

    except (Exception, e):
        print (str(e))

In [68]:
read_email_from_gmail(crs)

Connection OK
[b'1 2 3 4 5 6 7']
lakshay04014902016@msi-ggsip.org garglakshay631@gmail.com  ['jpg attachment', '', '-- ', 'Regards,', 'Lakshay Garg', 'Delhi']
lakshay04014902016@msi-ggsip.org garglakshay631@gmail.com  {'filename': 'download-imac-pro-wallpaper.jpg', 'payload': '...\r\nKUpSlKUpSlKUv+S3FLi4uL//AKzf/9k=', 'binary': True, 'mail_content_type': 'image/jpeg', 'content-id': '<f_jmce5c2m0>', 'content_transfer_encoding': 'base64'} <class 'dict'>


lakshay.garg1701@gmail.com garglakshay631@gmail.com test ['are you excited for MPR?', '', 'Lakshay 9711085333', 'Jayesh 98xxxxxxxx', '', 'department', 'CSE']
lakshay.garg1701@gmail.com garglakshay631@gmail.com test Please ask the organizer to send the poster
lakshay04014902016@msi-ggsip.org garglakshay631@gmail.com  ['Hi', '', '-- ', 'Regards,', 'Lakshay Garg', 'Delhi']
lakshay04014902016@msi-ggsip.org garglakshay631@gmail.com  Please ask the organizer to send the poster
lakshay04014902016@msi-ggsip.org garglakshay631@gmail.com Fwd: 2-Step Verification turned off ['---------- Forwarded message ---------', 'From: Google <no-reply@accounts.google.com>', 'Date: Tue, Sep 18, 2018 at 4:45 PM', 'Subject: 2-Step Verification turned off', 'To: <lakshay04014902016@msi-ggsip.org>', '', '', '2-Step Verification turned off', 'Hi lakshay,', 'You recently turned off 2-Step Verification for your Google Account', 'lakshay04014902016@msi-ggsip.org. This means that when you (or anyone else)', 'tries to 

googlecommunityteam-noreply@google.com garglakshay631@gmail.com Lakshay, welcome to your new Google Account ['Hi Lakshay,', '', 'Thank you for creating a Google Account. Here is some advice to get started  ', 'with your Google account.', '', 'Security  ', '<https://www.google.com/appserve/mkt/p/AL8lKjOkzOT1mtrfxyqo3aF4upFHQl0ZPQ6LHljeXHu_HjWle5RnxB7xDaQ22dfB1gdZKO-2vIRMDZIIRzS9AJmAj_N-XXFRR3v-SaHyFyTFzfGNz1gPxAF_aNJYPTxxrsXMBlq9C6ToxGlQH1auMHZeQXjX9NH3ug>', '', "You're in control", '', "Choose what's right for you. Manage your privacy and security settings at  ", 'any time in your Google Account  ', '<https://www.google.com/appserve/mkt/p/AL8lKjPbNLcHWYsTKifO0QGDkR2FQNPaQ8qCrahfhXugg2-E7Vjg9HFA-zSrk_zgm-jXrPWj9S1lFvEWOQZKvMTxXqA76HbSMo0LXK8r8GHDvtk4CAWMwFFYXahx7RXIN6_SMCl8Z_EaT70itA4PTD1nCEOMUjO0UOHGbXQD>,  ', 'where you can sign up for Privacy Checkup  ', '<https://www.google.com/appserve/mkt/p/AL8lKjPRcTylnbSrQJsZxAPCw26KGZEj1V1Blm6OijFDz4Pj2TpLWkXaKVbmqDQirZigMVq4tdHw8kg4_aGN5lQAyDD

In [63]:
conn=mysql.connector.connect(host="localhost",user="root",password="abcd@1234")
crs=conn.cursor()        

In [35]:
crs.execute('show databases')
for x in crs:
    print(x)

('information_schema',)
('MPR',)
('mysql',)
('performance_schema',)
('sys',)


In [36]:
crs.execute('use mpr')
crs.execute('show tables')
for x in crs:
    print(x)

In [27]:
crs.execute('drop table db')
crs.execute('describe db')
for x in crs:
    print(x)

ProgrammingError: 1146 (42S02): Table 'mpr.db' doesn't exist